**Maestría en Informática - PUCP - Aprendizaje Automático**

**Proyecto Grupal 2022-1**

# **Complicaciones en pacientes que sufrieron infarto al miocardio**

In [1]:
import pandas as pd

In [2]:
# Carga de datos - Pacientes que sufrieron infarto al miocardio y sus complicaciones
# Datos utilizados:
# Myocardial infarction complications Data Set
# Center for Machine Learning and Intelligent Systems, Bren School of Information and Computer Science, University of California, Irvine

mic_data = pd.read_csv('https://leicester.figshare.com/ndownloader/files/23581310')

In [3]:
mic_data.shape

(1700, 124)

In [4]:
# Levantamiento de información sobre las columnas

cant_col_nan      = 0
cant_col_sin_nan  = 0
cant_col_num      = 0
cant_col_binarias = 0

for col in mic_data:
  #Determinamos cuantas columna son numericas
  if pd.api.types.is_numeric_dtype(mic_data[col]):
    cant_col_num = cant_col_num + 1
    if mic_data[col].nunique() == 2:
      cant_col_binarias = cant_col_binarias + 1
  #Determinamos que columnas tienen datos faltantes
  if mic_data[col].isna().sum() > 0:
    cant_col_nan = cant_col_nan + 1
  else:
     cant_col_sin_nan = cant_col_sin_nan + 1

In [5]:
cant_col_mic_data = mic_data.shape[1]
print('Total de columnas en el conjunto de datos: ',cant_col_mic_data)
print('Total de columnas numericas:               ',cant_col_num, ' (', '%.1f%%' % (100*cant_col_num/cant_col_mic_data),')')
print('Total de columnas numericas binarias:       ',cant_col_binarias, ' (', '%.1f%%' % (100*cant_col_binarias/cant_col_mic_data),')')
print('Total de columnas con datos faltantes:     ',cant_col_nan, ' (', '%.1f%%' % (100*cant_col_nan/cant_col_mic_data),')')
print('Total de columnas con datos completos:      ',cant_col_sin_nan, ' (Para comprobación)')

Total de columnas en el conjunto de datos:  124
Total de columnas numericas:                124  ( 100.0% )
Total de columnas numericas binarias:        89  ( 71.8% )
Total de columnas con datos faltantes:      110  ( 88.7% )
Total de columnas con datos completos:       14  (Para comprobación)


Todas las columnas son numéricas. Algunas son enteras y otras de punto flotante.

89 columnas (71.8%) son binarias, es decir, tiene los valores 0 y 1. Por tanto 35 columan tiene datos no binarios.

La mayoría de las columnas tienen datos faltantes (88.7%).

In [ ]:
# Recoleccion de características de los datos por columna: Datos unicos, datos faltantes
cant_filas_mic_data = mic_data.shape[0]
mic_carat = pd.DataFrame(mic_data.nunique(),columns=['Datos_Unicos'])
mic_carat['Faltantes'] = mic_data.isna().sum() / cant_filas_mic_data * 100
# mic_data.nunique().hist()
# mic_data.nunique()

In [ ]:
print ('Columnas con mayor cantidad de datos únicos')
print ()
mic_carat.sort_values('Datos_Unicos', ascending = False)['Datos_Unicos'].head(37)

Columnas con mayor cantidad de datos únicos



ID             1700
L_BLOOD         174
ALT_BLOOD        69
AGE              62
ROE              58
AST_BLOOD        58
K_BLOOD          51
NA_BLOOD         40
S_AD_ORIT        32
S_AD_KBRIG       30
D_AD_KBRIG       21
D_AD_ORIT        20
TIME_B_S          9
LET_IS            8
DLIT_AG           8
STENOK_AN         7
post_im           5
inf_im            5
lat_im            5
ant_im            5
NA_R_1_n          5
NOT_NA_1_n        5
ZSN_A             5
FK_STENOK         5
KFK_BLOOD         4
R_AB_2_n          4
R_AB_1_n          4
NA_R_2_n          4
NOT_NA_2_n        4
INF_ANAM          4
R_AB_3_n          4
GB                4
NOT_NA_3_n        3
NA_R_3_n          3
IBS_POST          3
NA_KB             2
fibr_ter_06       2
Name: Datos_Unicos, dtype: int64

In [ ]:
print ('Columnas con mayor porcentaje de datos faltantes')
print ()
mic_carat.sort_values('Faltantes', ascending = False)['Faltantes'].head(15)

Columnas con mayor porcentaje de datos faltantes



KFK_BLOOD     99.764706
IBS_NASL      95.764706
S_AD_KBRIG    63.294118
D_AD_KBRIG    63.294118
NOT_NA_KB     40.352941
LID_KB        39.823529
NA_KB         38.647059
GIPER_NA      22.058824
NA_BLOOD      22.058824
K_BLOOD       21.823529
GIPO_K        21.705882
AST_BLOOD     16.764706
ALT_BLOOD     16.705882
S_AD_ORIT     15.705882
D_AD_ORIT     15.705882
Name: Faltantes, dtype: float64

De acuerdo a algunos autores, la proporción de valores faltantes en estudios científicos es muy variable y puede alcanzar niveles de entre 26% to 72% para estudios sobre educación y psicología, siendo valores comunes entre 15% to 20%.

Yiran Dong, Chao-Ying Joanne Peng, Principled missing data methods for researchers, Springerplus. 2013; 2: 222. Published online 2013 May 14. doi: 10.1186/2193-1801-2-222
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3701793/

Notas diversas encontradas en foros de internet sugieren valores entre 25% y 30% como aceptables.

Para el presente trabajo descartaremos las columnas con más de 25% de datos faltantes.

Las columnas a descartar serán las siguientes:

Columna       Porcent. faltantes   Información que contiene

---------     ------------------   ------------------------

KFK_BLOOD           99.8%          Serum CPK content

IBS_NASL            95.8%          Heredity on CHD

S_AD_KBRIG          63.3%          Systolic blood pressure according to Emergency Cardiology Team

D_AD_KBRIG          63.3%          Diastolic blood pressure according to Emergency Cardiology Team

NOT_NA_KB           40.4%         Use of NSAIDs by the Emergency Cardiology Team

LID_KB              39.8%         Use of lidocaine by the Emergency Cardiology Team

NA_KB               38.6%         Use of opioid drugs by the Emergency Cardiology Team


In [ ]:
filas_nan = 0
filas_sin_nan = 0
for i in range(mic_data.shape[0]):
  if mic_data.iloc[i].isna().sum() > 0:
    filas_nan = filas_nan + 1
  else:
    filas_sin_nan = filas_sin_nan + 1
filas_nan,filas_sin_nan

(1700, 0)